# Consigna del desafío 2
- Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.

- Probar términos de interés y explicar
similitudes en el espacio de embeddings.

- Intentar plantear y probar tests de
analogías.
- Graficar los embeddings
resultantes.
- Sacar conclusiones.

# Desarrollo del desafio 2

Con el comando pip install datasets se instala la librería datasets de Hugging Face en tu entorno de Python. Esta librería es ampliamente utilizada en el campo del procesamiento de lenguaje natural (NLP) y el aprendizaje automático (ML) para descargar, manipular y preprocesar conjuntos de datos grandes.

In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Cargamos librerías

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
from datasets import load_dataset

SST-2 (Stanford Sentiment Treebank 2):

Es uno de los conjuntos de datos incluidos en GLUE y está diseñado específicamente para el análisis de sentimientos.
SST-2 consiste en frases cortas extraídas de reseñas de películas que están etiquetadas como positivas o negativas, lo que permite entrenar modelos para clasificar el sentimiento de un texto.
Es particularmente útil para medir cómo los modelos comprenden y diferencian el tono o la polaridad de un texto breve.

In [3]:
# Cargar el dataset GLUE subset sst-2
dataset = load_dataset("glue", "sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
df= pd.DataFrame(dataset['train'])


In [5]:
len( df )

67349

In [6]:
df = pd.DataFrame(dataset['train']['sentence'], columns=["sentence"])
df.drop_duplicates(inplace=True)
df.shape

(66978, 1)

In [7]:
# Ver las primeras filas del DataFrame
df.sample(10)

,sentence
44692,those prone to indignation need not apply
24787,"thanks to a small star with big heart , this f..."
47583,has made literature literal without killing it...
51819,kindness
27032,in the end it 's as sweet as greenfingers
197,enables shafer to navigate spaces both large ....
29547,seem to get a coherent rhythm going
64602,experimental enough to alienate the mainstream...
58174,the biggest problem i have ( other than the ve...
54398,offers the none-too-original premise that ever...


### Preprocesamiento

In [8]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-8-99a399a8b871>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [9]:
# Demos un vistazo
sentence_tokens[:2]

[['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units'],
 ['contains', 'no', 'wit', 'only', 'labored', 'gags']]

### Creamos los vectores Word2Vec

In [10]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [11]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)

In [12]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [13]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 66978


Se muestran la cantidad de palabras distintas en el vocabulario.

In [14]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 10640


### Entrenamos los embedinngs

In [15]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 4584717.5
Loss after epoch 1: 3020301.0
Loss after epoch 2: 2635128.5
Loss after epoch 3: 2305592.0
Loss after epoch 4: 2073328.0
Loss after epoch 5: 1905097.0
Loss after epoch 6: 1623752.0
Loss after epoch 7: 1517182.0
Loss after epoch 8: 1453620.0
Loss after epoch 9: 1414450.0
Loss after epoch 10: 1379774.0
Loss after epoch 11: 1351130.0
Loss after epoch 12: 1322840.0
Loss after epoch 13: 1299532.0
Loss after epoch 14: 1282370.0
Loss after epoch 15: 1262120.0
Loss after epoch 16: 1241966.0
Loss after epoch 17: 1226870.0
Loss after epoch 18: 1081826.0
Loss after epoch 19: 917680.0


(9179084, 11966160)

###Prueba de analogías

In [16]:
result = w2v_model.wv.most_similar(positive=['love', 'fantasy'], negative=['war'], topn=1)
print(result)

[('bawdy', 0.41554296016693115)]


El modelo ha aprendido que la palabra "bawdy" tiene una relación contextual con las palabras "love" y "fantasy". Dado que el dataset esta bajo el contexto de comentarios de películas.

In [17]:
result = w2v_model.wv.most_similar(positive=['nuts', 'crazy'], negative=['love'], topn=1)
print(result)

[('conspiracy', 0.5495546460151672)]


El modelo ha logrado capturar la relación entre la palabara "conspiracy", con las palabras "nuts" y "crazy" que son muy probables de encontrar en reseñas de películas de estas características.

### Análisis de similitud entre palabras

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["conspiracy"], topn=10)

[('nuts', 0.8248449563980103),
 ('jfk', 0.7722594738006592),
 ('speculation', 0.6845639944076538),
 ('stone', 0.6746784448623657),
 ('dass', 0.6645463705062866),
 ('ram', 0.6423006653785706),
 ('theories', 0.6418635249137878),
 ('drill', 0.6392623782157898),
 ('fairytale', 0.6339572072029114),
 ('nightmarish', 0.6288698315620422)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["war"], topn=10)

[('degree', 0.02654319256544113),
 ('sensitive', 0.018068021163344383),
 ('fantasy', 0.014436752535402775),
 ('episodes', 0.002560412511229515),
 ('excellent', 0.001951520680449903),
 ('chemistry', 0.0015428500482812524),
 ('bite', -0.00011322957288939506),
 ('delightful', -0.0009954982670024037),
 ('writer', -0.001718792482279241),
 ('onscreen', -0.0036785590928047895)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bawdy"], topn=10)

[('farcically', 0.9658797383308411),
 ('mcbeal', 0.7922398447990417),
 ('melanie', 0.790252149105072),
 ('pablum', 0.7688817381858826),
 ('ballerinas', 0.750948429107666),
 ('16', 0.7497775554656982),
 ('overstylized', 0.7496245503425598),
 ('upheaval', 0.7450026273727417),
 ('puréed', 0.7410769462585449),
 ('enervating', 0.7405593991279602)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["war"], topn=10)

[('degree', 0.02654319256544113),
 ('sensitive', 0.018068021163344383),
 ('fantasy', 0.014436752535402775),
 ('episodes', 0.002560412511229515),
 ('excellent', 0.001951520680449903),
 ('chemistry', 0.0015428500482812524),
 ('bite', -0.00011322957288939506),
 ('delightful', -0.0009954982670024037),
 ('writer', -0.001718792482279241),
 ('onscreen', -0.0036785590928047895)]

Cuando no existe la palabra se lanza un error que nos informa que la palabra no se encuentra en el vocabulario

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["dietyyy"])

KeyError: "Key 'dietyyy' not present in vocabulary"

In [21]:
# el método `get_vector` permite obtener los vectores:
vector_conspiracy = w2v_model.wv.get_vector("conspiracy")
print(vector_conspiracy)

[-0.09941504  0.21755952  0.07784607 -0.19316886 -0.25846606  0.233652
  0.10504859  0.24318959  0.6986296   0.26292777 -0.08570158 -0.4988388
 -0.17287047 -0.49071124 -0.2774305  -0.3897657   0.06720262 -0.41720024
  0.31129065 -0.47631794 -0.2854257   0.21244368 -0.04506661 -0.01962329
  0.13005757  0.00467317 -0.43655282 -0.10530818  0.18442172 -0.07491179
 -0.22508772  0.06652967 -0.30635706  0.16235733  0.1375249  -0.13048543
 -0.21753523 -0.05538192  0.5157433  -0.03258669  0.06069343  0.20875907
 -0.03126958  0.112308   -0.26341516  0.33516535 -0.02986848  0.21270953
  0.43192792 -0.06588848  0.10822544 -0.28237784  0.34844613  0.46506047
 -0.538552   -0.13792606  0.01809986 -0.27202845  0.3327591  -0.18495469
  0.16449226 -0.06997392  0.17978182 -0.30989453 -0.66589546 -0.3580008
  0.19846253 -0.0289483   0.08813174  0.01549579 -0.70665544  0.21766232
  0.53370374 -0.4484514  -0.07548043  0.13437419 -0.4206096  -0.12261522
 -0.2670299   0.21178965 -0.14203882  0.534573    0.014

In [22]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_conspiracy)

[('conspiracy', 1.0),
 ('nuts', 0.8248449563980103),
 ('jfk', 0.7722594738006592),
 ('speculation', 0.6845639944076538),
 ('stone', 0.6746784448623657),
 ('dass', 0.6645463705062866),
 ('ram', 0.6423006653785706),
 ('theories', 0.6418635249137878),
 ('drill', 0.6392623782157898),
 ('fairytale', 0.6339572072029114)]

### Visualización de relaciones entre palabras

In [31]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [33]:
#Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

###Podemos guardar los vectores y labels como tsv para graficar en http://projector.tensorflow.org/

In [ ]:

# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)